# Estimate errors

In [1]:
# Local path, change this.
yellow = 'C:/Users/caspe/Desktop/yellow/'
yellow_lib = yellow + "lib/"

import sys; sys.path.append(yellow_lib) 
import sqlite3
import pandas as pd
import ml_utils
from os import path
from glob import glob
import re
import sys
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_squared_error
from math import sqrt

In [2]:
def get_tablename(db_connection):
    query = "SELECT name FROM sqlite_master WHERE type IN ('table') AND name NOT LIKE 'sqlite_%' ORDER BY 1;"
    table_name = pd.read_sql_query(query, db_connection)
    return table_name.values[0][0]

def clean_cols(df):
    replace_map = {}
    for col in df.columns:
        replace_map[col] = re.sub("\ |\,|\)|\(|\(|\'", "", col)
    return df.rename(columns=replace_map)

In [3]:
folder = "C:/Users/caspe/Desktop/yellow/papers/structure_density/predictions/bsac_bsdc_s2/"

results = glob(folder + '*.sqlite')
# result = results[0]

for result in results:
    db_cnx = sqlite3.connect(result)
    table_name = get_tablename(db_cnx)
    df = pd.read_sql_query(f"SELECT * FROM '{table_name}';", db_cnx)
    df = clean_cols(df)

    filename = os.path.splitext(os.path.basename(result))[0].split('_')
    target = filename[2]
    if target == "vol": target = "volume"
    test_area = filename[3]

    table_name_adj = "_".join(table_name.split("_")[:2] + table_name.split("_")[-2:])

    res = pd.DataFrame({ "pred": df[table_name_adj].values, "target": df[target].values })

    mae = mean_absolute_error(res["target"], res["pred"])
    meae = median_absolute_error(res["target"], res["pred"])
    abs_percentage_error = abs(res["target"] - res["pred"]) / (abs(res["target"] + sys.float_info.epsilon))
    meape = (abs_percentage_error * 100).median()
    r2 = r2_score(res["target"], res["pred"])
    rmse = sqrt(mean_squared_error(res["target"], res["pred"]))

    print(f"    Test area: {test_area} - {target}")
    print(f"    Mean Absolute Error (MAE):                {ml_utils.pad(str(round(mae, 2)), 5, 2)}")
    print(f"    Median Absolute Error (MeAE):             {ml_utils.pad(str(round(meae, 2)), 5, 2)}")
    print(f"    Median Absolute Percentage Error (MAPE):  {ml_utils.pad(str(round(meape, 2)), 5, 2)}")
    print(f"    R Squared:                                {ml_utils.pad(str(round(r2, 3)), 5, 3)}")
    print(f"    Root Mean Squared Error:                  {ml_utils.pad(str(round(rmse, 2)), 5, 2)}")
    print("")

    Test area: lemvig - area
    Mean Absolute Error (MAE):                   81.33
    Median Absolute Error (MeAE):                 0.00
    Median Absolute Percentage Error (MAPE):      0.00
    R Squared:                                    0.877
    Root Mean Squared Error:                    305.30

    Test area: aarhus - volume
    Mean Absolute Error (MAE):                 1947.88
    Median Absolute Error (MeAE):                38.66
    Median Absolute Percentage Error (MAPE):      1.14
    R Squared:                                    0.821
    Root Mean Squared Error:                   6931.14

    Test area: lemvig - volume
    Mean Absolute Error (MAE):                  400.74
    Median Absolute Error (MeAE):                 0.00
    Median Absolute Percentage Error (MAPE):      0.00
    R Squared:                                    0.790
    Root Mean Squared Error:                   1841.26

    Test area: silkeborg - volume
    Mean Absolute Error (MAE):              